In [1]:
include("smc_model.jl")
include("smc_model_parameters.jl")
using DifferentialEquations
using LaTeXStrings
using DataFrames
using Plots
using Measures
using DataStructures
using NLsolve
using WAV
using Dierckx 
using FFTW
using DSP
using Statistics
using JLD2, FileIO
using ProgressMeter
gr()

q0 = Control_params();
p0 = Fixed_params();
s0 = State();

# Construct and solve elevated systems

In [18]:

function simulate_rescaled(s_er, s_mit, s_ecs)
    q0_dict = type2dict(q0);
    s0_dict = type2dict(s0);

    delete!(q0_dict,Symbol("g_leak_mit"));
    q0_dict[Symbol("k_leak_er")] = 0.0;
    delete!(q0_dict,Symbol("g_leak_ecs"));

    delete!(q0_dict,Symbol("SERCA"));

    delete!(s0_dict,Symbol("IP3"));
    delete!(s0_dict,Symbol("IP3RX00"));
    delete!(s0_dict,Symbol("IP3RX10"));
    delete!(s0_dict,Symbol("IP3RX01"));
    delete!(s0_dict,Symbol("RyRR10"));
    delete!(s0_dict,Symbol("RyRR11"));
    delete!(s0_dict,Symbol("RyRR01"));
    delete!(s0_dict,Symbol("MyoMp"));
    delete!(s0_dict,Symbol("MyoAM"));
    delete!(s0_dict,Symbol("MyoAMp"));
    delete!(s0_dict,Symbol("G"));
    delete!(s0_dict,Symbol("PIP2"));
    delete!(s0_dict,Symbol("Rs"));
    delete!(s0_dict,Symbol("Rsp"));

    excludedODEs = map(v -> Symbol(v), ["Ca_ecs","Ca_er","Ca_cyt","Phi_ecs","Ca_mit_source","Ca_ecs_source"]);
    includedJs = map(v -> Symbol(v), ["er_cyt","ecs_cyt"]);
    s0_balanced_dict, q0_balanced_dict = balance(s0_dict,q0_dict,excludedODEs=excludedODEs,includedJs = includedJs);

    s0_balanced = reconstruct(s0,merge(s0_dict,s0_balanced_dict));
    q0_balanced = reconstruct(q0,merge(q0_dict,q0_balanced_dict));
    s0_elevated = reconstruct(s0_balanced, Ca_mit = 0.25);
    q0_elevated = reconstruct(q0_balanced, L=0.0);
    s0_elevated_dict = type2dict(s0_elevated);
    (output, problem, solution) = solveODEs(
        s0_elevated,q0_elevated,
        timespan=(0.0,60*20),
        fix = ["Ca_mit","Ca_ecs"]
        ,dtmax=10,saveat=1,rd_cer=1s_er,rd_cmi=s_mit,rd_ces=s_ecs);
    return output
end

simulate_rescaled (generic function with 1 method)

In [19]:
if isfile("rescaling_grid_sims.jld2")
    @load "rescaling_grid_sims.jld2" results
else
    pts = 20;

    log_alpha_vals = range(-1,1,length=pts);

    results = [];
    n = 1;
    p = Progress(pts^3, 2);
    for er in log_alpha_vals
        for mit in log_alpha_vals
            for ecs in log_alpha_vals
                push!(results,(er,mit,ecs,simulate_rescaled(10^er,10^mit,10^ecs)));
                ProgressMeter.update!(p, n)
                n += 1
            end
        end
    end
    @save "rescaling_grid_sims.jld2" results
end



Progress: 100%|█████████████████████████████████████████| Time: 7:44:24


In [20]:
function estimatefr(signal_o,fs;start=500)
    Ns = length(signal_o);
    if std(signal_o)!= 0.
        signal_n = (signal_o -mean(signal_o)*ones(Float16, (Ns,1)))/std(signal_o);
    else
        signal_n = (signal_o -mean(signal_o)*ones(Float16, (Ns,1)));
    end
    signal = signal_n[start:end];
    N = length(signal);
    r = periodogram(signal;nfft=N,fs=fs);
    p = r.power;
    p_sort =sortperm(abs.(p));
    freqArray = r.freq;
    freq_dom = freqArray[p_sort[end]];
    
    if freq_dom != 0
        period = 1/freq_dom;
        per_l = ceil(Int,period/fs);
    else
        per_l =  ceil(Int,N/4);
    end
    sig_p = signal_o[max(start,(end-2*per_l)):end];
 #   t_p = tti[end-2*per_l:end];
    p_max = maximum(sig_p);
    p_min = minimum(sig_p);
    #p_A = p_max-p_min;
    p_mean = mean(sig_p);

 return (freq_dom,p_max,p_min,p_mean)
    
end

function process_traces(df)
    dict = Dict()
    fs = 1;
    tti=0:1/fs: df[:t][end];
    for col in names(df)
        Ni =length(tti);
     # cyc 
        spl = Spline1D(df[:t], df[col]; k=2); 
        signal_o= spl(tti);
        dict[col] = estimatefr(signal_o,1; start = 300);
    end
    return dict
end

process_traces (generic function with 1 method)

In [21]:
processed = map(x -> (x[1],x[2],x[3],process_traces(x[4])), results );

In [22]:
oscillations = map(x -> [x[1],x[2],x[3], x[4][:Ca_cyt][1]>0], processed);

In [26]:
er = map(u -> u[1], processed);
mit = map(u -> u[2], processed);
ecs = map(u -> u[3], processed);
freq = map(u -> u[4][:Ca_cyt][1], processed);
Fr_mean = map(u -> u[4][:other_Fr][4], processed);
Ca_mean = map(u -> u[4][:Ca_cyt][4], processed);
J_ecs_mean = map(u -> u[4][:J_er_cyt][4], processed);
J_ecs_min = map(u -> u[4][:J_er_cyt][2], processed);
J_ecs_max = map(u -> u[4][:J_er_cyt][3], processed);
J_mit_mean = map(u -> u[4][:J_mit_cyt][4], processed);

In [27]:
summary_grid_data = DataFrame(er=er,mit=mit,ecs=ecs,freq=freq,
    Fr_mean=Fr_mean,Ca_mean=Ca_mean,
    J_ecs_mean=J_ecs_mean,J_mit_mean=J_mit_mean);

In [28]:
using CSV
CSV.write("summary_grid_data.csv",summary_grid_data)


"summary_grid_data.csv"

In [17]:
plt3d = plot(x[w.<0.5],y[w.<0.5],z[w.<0.5],seriestype=:scatter, alpha=0.05, color=:red, label="none")
plot!(x[w.>0.5],y[w.>0.5],z[w.>0.5],seriestype=:scatter, alpha=.3, color=:blue, label="oscillations")
plot!(xlim=(-1,1),ylim=(-1,1),zlim=(-1,1))
plot!(xlab=L"\log_{10} \alpha")
display(plt3d)


UndefVarError: UndefVarError: w not defined